# Summary

The following is the step taken to clean the datasets
- Removing duplicates based on 'raw_mail' column
- Change the "None" to an actuall None value to all column
- Remove email address that is not in the same format
- Fill empty/none email address with bfill and ffill
- Fill empty/none subject with bffill and ffill
- Update the 'date' so it is in one format
- Add malicious column

# import libraries

Load the libraries that are needed

In [23]:
import pandas as pd
from datetime import datetime
from dateutil import parser
import re

Load the datasets

In [24]:
fraudDataframe = pd.read_json('datasets/raw/fradulent_emails.json', orient='index')
phishingDataframe = pd.read_json('datasets/raw/phishing-chorpus.json', orient='index')
enronDataframe = pd.read_csv('datasets/raw/enron-emails.csv')

Remove duplicates

In [25]:
fraudDataframe = fraudDataframe.drop_duplicates(subset="raw_mail")
phishingDataframe = phishingDataframe.drop_duplicates(subset="raw_mail")
enronDataframe = enronDataframe.drop_duplicates(subset="raw_mail")

From a quick glance from all the 3 datasets, there are multiple inconsistensy that can be found in the format of the values. 

- from and to columns contains not only the emails
- datetime isn't in one format

In [7]:
print(fraudDataframe.isna().sum(), '\n') # contains none but isnt register as one
print(phishingDataframe.isna().sum(), '\n') # contains none but isnt register as one
print(enronDataframe.isna().sum()) # contains null values

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64 

Unnamed: 0         0
body               0
subject        19187
raw_mail           0
from               0
to             21847
status        517401
date               0
dtype: int64


As it show above, only the enron datasets contains a null values while in fact all 3 datasets does contains a "null" values

If we check the values for a "None" in a string format we will infact found that the rest of the datasets does in fact contains a Null value 

In [8]:
print((fraudDataframe == "None").sum())
print((phishingDataframe == "None").sum())
print((enronDataframe == "None").sum())

raw_mail      0
subject      17
from        365
to          948
status        0
date        534
body          0
dtype: int64
raw_mail     0
subject     49
from         4
to           9
status       5
date         3
body         0
dtype: int64
Unnamed: 0    0
body          0
subject       0
raw_mail      0
from          0
to            0
status        0
date          0
dtype: int64


Update the datasets to change the "None" values to an actual None, by applying a function to each column

In [26]:
def updateToNone(val):
    if val == "None":
        return None
    else:
        return val

fraudDataframe['subject'] = fraudDataframe['subject'].apply(updateToNone)
fraudDataframe['to'] = fraudDataframe['to'].apply(updateToNone)
fraudDataframe['from'] = fraudDataframe['to'].apply(updateToNone)
fraudDataframe['status'] = fraudDataframe['status'].apply(updateToNone)
fraudDataframe['date'] = fraudDataframe['date'].apply(updateToNone)

print((fraudDataframe == "None").sum())
print(fraudDataframe.isna().sum()) 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64
raw_mail      0
subject      17
from        948
to          948
status        0
date        534
body          0
dtype: int64


In [27]:
phishingDataframe['subject'] = phishingDataframe['subject'].apply(updateToNone)
phishingDataframe['to'] = phishingDataframe['to'].apply(updateToNone)
phishingDataframe['from'] = phishingDataframe['to'].apply(updateToNone)
phishingDataframe['status'] = phishingDataframe['status'].apply(updateToNone)
phishingDataframe['date'] = phishingDataframe['date'].apply(updateToNone)

print((fraudDataframe == "None").sum())
print(fraudDataframe.isna().sum()) 

raw_mail    0
subject     0
from        0
to          0
status      0
date        0
body        0
dtype: int64
raw_mail      0
subject      17
from        948
to          948
status        0
date        534
body          0
dtype: int64


To fix the inconsistensy format of the email in columns "from" and "to" , we will used regex to extract the valid emails first then, fill the empty values with valid values from the datasets

the pattern for capturing a valid email is "[a-zA-Z0-9-_.]*@a-zA-Z0-9-]*(\\.[a-zA-Z]*)*"

the patter will look for email that have a valid username + @ + domain

In [28]:
notValidEmail = phishingDataframe['from'].str.contains('[a-zA-Z0-9-_.]*@a-zA-Z0-9-]*(\.[a-zA-Z]*)*', regex=True) == False
phishingDataframe[notValidEmail]['from'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_18780\3334997152.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = phishingDataframe['from'].str.contains('[a-zA-Z0-9-_.]*@a-zA-Z0-9-]*(\.[a-zA-Z]*)*', regex=True) == False


4181

In [29]:
notValidEmail = fraudDataframe['from'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False
fraudDataframe[notValidEmail]['from'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_18780\1799348055.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = fraudDataframe['from'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False


545

In [30]:
notValidEmail = enronDataframe['to'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False
enronDataframe[notValidEmail]['to'].count()

C:\Users\candi\AppData\Local\Temp\ipykernel_18780\2401131589.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  notValidEmail = enronDataframe['to'].str.contains('([a-zA-Z0-9-_.])*@([a-zA-Z0-9-])*(\.[a-zA-Z]*)*', regex=True) == False


16

the 3 cells above shows the number of time that a row match pattern

In [31]:
parsedFrom = fraudDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = fraudDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

fraudDataframe['parsed_from'] = parsedFrom[0]
fraudDataframe['parsed_to'] = parsedTo[0]

In [15]:
parsedFrom = phishingDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = phishingDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

phishingDataframe['parsed_from'] = parsedFrom[0]
phishingDataframe['parsed_to'] = parsedTo[0]

In [16]:
parsedFrom = enronDataframe['from'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')
parsedTo = enronDataframe['to'].str.extract('([a-zA-Z0-9-_.]*@[a-zA-Z0-9-]*(\.[a-zA-Z]*)*)')

enronDataframe['parsed_from'] = parsedFrom[0]
enronDataframe['parsed_to'] = parsedTo[0]

Bellow is the kind of fields that wasnt register as an email

In [17]:
print(fraudDataframe[fraudDataframe['parsed_from'].isna()]['from'].unique())
print(fraudDataframe[fraudDataframe['parsed_to'].isna()]['to'].unique())
print(phishingDataframe[phishingDataframe['parsed_from'].isna()]['from'].unique())
print(phishingDataframe[phishingDataframe['parsed_to'].isna()]['to'].unique())

[None 'undisclosed-recipients: ;' 'undisclosed-recipients:;' ''
 'undisclosed recipients: ;' 'N/A <>, N/A <>' 'N/A <>']
[None 'undisclosed-recipients: ;' 'undisclosed-recipients:;' ''
 'undisclosed recipients: ;' 'N/A <>, N/A <>' 'N/A <>']
['undisclosed-recipients: ;' '[removed]' None 'undisclosed-recipients:;'
 'unlisted-recipients:; (no To-header on input)'
 '<Undisclosed-Recipient:;>' '=?euc-kr?B?u+e2+7nnu/W6rsbtwfawocG3?=' '']
['undisclosed-recipients: ;' '[removed]' None 'undisclosed-recipients:;'
 'unlisted-recipients:; (no To-header on input)'
 '<Undisclosed-Recipient:;>' '=?euc-kr?B?u+e2+7nnu/W6rsbtwfawocG3?=' '']


In [18]:
phishingDataframe['parsed_from'] = phishingDataframe['parsed_from'].ffill().bfill()
fraudDataframe['parsed_from'] = fraudDataframe['parsed_from'].ffill().bfill()
enronDataframe['parsed_from'] = enronDataframe['parsed_from'].ffill().bfill()

In [19]:
phishingDataframe['parsed_to'] = phishingDataframe['parsed_to'].ffill().bfill()
fraudDataframe['parsed_to'] = fraudDataframe['parsed_to'].ffill().bfill()
enronDataframe['parsed_to'] = enronDataframe['parsed_to'].ffill().bfill()

all the datasets subject column contains a null value, we will fill this value using existing fields in the datasets

In [20]:
phishingDataframe['subject'] = phishingDataframe.subject.ffill().bfill()
fraudDataframe['subject'] = fraudDataframe.subject.ffill().bfill()
enronDataframe['subject'] = enronDataframe.subject.ffill().bfill()

In [21]:
print('Number of row that have empty subject for phishingDataframe:', phishingDataframe.subject.isnull().sum())
print('Number of row that have empty subject for fraudDataframe:', fraudDataframe.subject.isnull().sum())
print('Number of row that have empty subject for enronDataframe:', enronDataframe.subject.isnull().sum())

Number of row that have empty subject for phishingDataframe: 0
Number of row that have empty subject for fraudDataframe: 0
Number of row that have empty subject for enronDataframe: 0


In [32]:
enronDataframe['parsed_date'] = enronDataframe.date.apply(lambda date: parser.parse(date).isoformat())

Fill empty fields so no null exist by doing backward and forward fill

In [33]:
fraudDataframe.date = fraudDataframe.date.ffill().bfill()
phishingDataframe.date = phishingDataframe.date.ffill().bfill()

In [34]:
diff = phishingDataframe.shape[0] - phishingDataframe.date.str.contains('[A-Za-z]{0,3}, \d* [A-Za-z]{0,3} \d{4}').sum()
print("Total date row that are not in format for phishingDataframe:", diff)
diff = fraudDataframe.shape[0] - fraudDataframe.date.str.contains('[A-Za-z]{0,3}, \d* [A-Za-z]{0,3} \d{4}').sum()
print("Total date row that are not in format for phishingDataframe:", diff)

Total date row that are not in format for phishingDataframe: 444
Total date row that are not in format for phishingDataframe: 151


In [35]:
def parseDate(date):
    try:
        return parser.parse(date).isoformat()
    except Exception as e:
        return None

In [36]:
phishingDataframe['parsed_date'] = phishingDataframe.date.str.replace('\.', ':', regex=True)
phishingDataframe['parsed_date'] = phishingDataframe['parsed_date'].apply(parseDate)

c:\Users\candi\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Manual cleaning for cases that are to few to automate

In [37]:
phishingDataframe.loc[821].parsed_date = parser.parse("Fri, 09 Jun 2006 08:23:29 +0500 (EST)").isoformat()
phishingDataframe.loc[892].parsed_date = parser.parse("Fri, 23 Jun 2006 13:25:46 -0100 (EST)").isoformat()
phishingDataframe.loc[896].parsed_date = parser.parse("Fri, 23 Jun 2006 21:36:05 +0800").isoformat()
phishingDataframe.loc[1066].parsed_date = parser.parse("Wed, 26 Jul 2006 09:48:28 -0800").isoformat()
phishingDataframe.loc[1067].parsed_date = parser.parse("Wed, 26 Jul 2006 12:50:48 -0600").isoformat()
phishingDataframe.loc[1072].parsed_date = parser.parse("Thu, 27 Jul 2006 03:06:10 -0800").isoformat()
phishingDataframe.loc[1074].parsed_date = parser.parse("Wed, 26 Jul 2006 15:24:52 -0500").isoformat()
phishingDataframe.loc[1075].parsed_date = parser.parse("Wed, 26 Jul 2006 15:43:42 -0500").isoformat()
phishingDataframe.loc[1076].parsed_date = parser.parse("Wed, 26 Jul 2006 19:03:49 -0300").isoformat()
phishingDataframe.loc[1077].parsed_date = parser.parse("Wed, 26 Jul 2006 19:35:02 -0300").isoformat()
phishingDataframe.loc[1095].parsed_date = parser.parse("31.07.2006").isoformat()
phishingDataframe.loc[1173].parsed_date = parser.parse("Thu, 3 Aug 2006 00:13:00 -0530").isoformat()
phishingDataframe.loc[2421].parsed_date = parser.parse("Tue, 09 Jan 2007 14:00:44 +0430").isoformat()
phishingDataframe.loc[3540].parsed_date = parser.parse("Sun, 10 Sep 2006 14:00:47 +0000").isoformat()
phishingDataframe.loc[3643].parsed_date = parser.parse("Fri, 09 Mar 2007 18:11:57 +0530").isoformat()
phishingDataframe.loc[3896].parsed_date = parser.parse("07.08.2006").isoformat()
phishingDataframe.loc[3963].parsed_date = parser.parse("Mon, 24 Feb 2003 17:32:08 +0000").isoformat()
phishingDataframe.loc[4117].parsed_date = parser.parse("Sun, 10 Sep 2006 12:08:54 -0300").isoformat()

In [38]:
fraudDataframe['parsed_date'] = fraudDataframe.date.str.replace('\.', ':', regex=True)
fraudDataframe['parsed_date'] = fraudDataframe['parsed_date'].apply(parseDate)

c:\Users\candi\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname BST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [39]:
def myfunc(row):
    if row.parsed_date == None:
        try:
            row.parsed_date = parser.parse(
                re.search("([A-Za-z]{1,3}, \d{0,2} [A-Za-z]* \d{2,4} \d{2}:\d{2}:\d{2} ((\+|\-)?\d{4})?)", 
                          row.date).group(1)).isoformat()
            return row
        except Exception as e:
            return row
    else:
        return row

fraudDataframe = fraudDataframe.apply(myfunc, axis=1)

Manual Updates for cases that are to few to automate

In [40]:
fraudDataframe.loc[542].parsed_date = parser.parse("Sun, 09 nov 2003 21:18:28").isoformat()
fraudDataframe.loc[1236].parsed_date = parser.parse("Tue, 09 nov 2004 15:38:35 -0300").isoformat()

Add prediction label that will be used in model training

In [41]:
fraudDataframe['malicious'] = True
phishingDataframe['malicious'] = True
enronDataframe['malicious'] = False

In [42]:
fraudDataframe.to_csv(path_or_buf='datasets/clean/fraud-emails.csv', index=False)
phishingDataframe.to_csv(path_or_buf='datasets/clean/phishing-emails.csv', index=False)
enronDataframe.to_csv(path_or_buf='datasets/clean/enron-emails.csv', index=False)